In [1]:
import json
import tensorflow as tf
import numpy as np 
import pandas as pd

2025-08-27 05:03:12.531515: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756270992.711018      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756270992.759744      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
data = []
filename = "/kaggle/input/dataset/hinglish_upload_v1.json"

with open(filename,"r",encoding="utf-8") as f:
    for line in f:
        obj = json.loads(line)
        data.append({
            "English": obj["translation"]["en"],
            "Hinglish": obj["translation"]["hi_ng"]
        })

In [3]:
df = pd.DataFrame(data)
df.head()

,English,Hinglish
0,What's the name of the movie,film ka kya naam hai
1,"Hi, the rotten tomatoes score is great but the...","namaste, sada hua tomatoes score mahaan hai, l..."
2,Do you think you will like the movie,kya aapako lagata hai ki aapako film pasand aa...
3,What kind of movie is it,yah kis tarah kee philm hai
4,when was the movie made?,film kab banee thee?


In [4]:
df.describe()

,English,Hinglish
count,189102,189102
unique,165608,155544
top,cancel all alarms,alarm ko stop kare
freq,64,88


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 189102 entries, 0 to 189101
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   English   189102 non-null  object
 1   Hinglish  189102 non-null  object
dtypes: object(2)
memory usage: 2.9+ MB


In [6]:
df.isnull().sum()

English     0
Hinglish    0
dtype: int64

In [7]:
df_final = df.sample(frac=1).reset_index(drop=True)

In [8]:
df_final.head()

,English,Hinglish
0,Show my reminders for Tuesday,Tuesday ke liye mere reminders ko dikhao
1,Formal events tonight,aj raat Formal events hai
2,Will it rain tomorrow ?,kya kal barish hone wali hai ?
3,Weather update .,Weather ka update .
4,Lower the volume,volume ko lower kare


In [9]:
en_sentence = df_final["English"]
hing_sentence = df_final["Hinglish"]
print(en_sentence.head())
print(hing_sentence.head())

0    Show my reminders for Tuesday
1            Formal events tonight
2          Will it rain tomorrow ?
3                 Weather update .
4                 Lower the volume
Name: English, dtype: object
0    Tuesday ke liye mere reminders ko dikhao
1                   aj raat Formal events hai
2              kya kal barish hone wali hai ?
3                         Weather ka update .
4                        volume ko lower kare
Name: Hinglish, dtype: object


In [10]:
def preprocess_text(text):
    text = str(text).lower().strip()
    return text

In [11]:
en_sentence = en_sentence.apply(preprocess_text)
hing_sentence = hing_sentence.apply(preprocess_text)

In [12]:
vocab_size = 4000
max_length = 50
text_vec_layer_en = tf.keras.layers.TextVectorization(
    vocab_size,output_sequence_length=max_length,
    standardize="lower_and_strip_punctuation"
) 
text_vec_layer_hing = tf.keras.layers.TextVectorization(
    vocab_size,output_sequence_length=max_length,
    standardize="lower_and_strip_punctuation"
) 

I0000 00:00:1756271006.902082      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


In [13]:
text_vec_layer_en.adapt(en_sentence)
text_vec_layer_hing.adapt([f"sos {s} eos" for s in hing_sentence])

In [14]:
print(f"English vocab size: {len(text_vec_layer_en.get_vocabulary())}")
print(f"Hindi vocab size: {len(text_vec_layer_hing.get_vocabulary())}")


English vocab size: 4000
Hindi vocab size: 4000


In [15]:
vocab_en = text_vec_layer_en.get_vocabulary()
vocab_en = [str(word) for word in vocab_en]
print(vocab_en[:10])

['', '[UNK]', 'the', 'to', 'for', 'is', 'i', 'my', 'in', 'me']


In [16]:
vocab_hing = text_vec_layer_hing.get_vocabulary()
vocab_hing = [str(word) for word in vocab_hing]
print(vocab_hing[:10])

['', '[UNK]', 'sos', 'eos', 'ke', 'liye', 'hai', 'ko', 'kya', 'me']


In [17]:
X_seq_enc = text_vec_layer_en(en_sentence)
X_seq_dec = text_vec_layer_hing([f"sos {s}" for s in hing_sentence])
Y_seq = text_vec_layer_hing([f"{s} eos" for s in hing_sentence])

In [18]:
X_train_enc = X_seq_enc[:150_000]
X_valid_enc = X_seq_enc[150_000:]
X_train_dec = X_seq_dec[:150_000]
X_valid_dec = X_seq_dec[150_000:]
Y_train = Y_seq[:150_000]
Y_valid = Y_seq[150_000:]

In [19]:
Y_train.shape

TensorShape([150000, 50])

In [20]:
Y_train = np.expand_dims(Y_train,-1)
Y_train.shape

(150000, 50, 1)

In [21]:
def create_model():
    encoder_inputs = tf.keras.layers.Input(shape=(max_length,),dtype=tf.int32)
    decoder_inputs = tf.keras.layers.Input(shape=(max_length,),dtype=tf.int32)
    
    embed_size = 128
    encoder_embeddings = tf.keras.layers.Embedding(vocab_size,embed_size,
                                                   mask_zero=True)(encoder_inputs)
    encoder_embeddings = tf.keras.layers.Dropout(0.1)(encoder_embeddings)
    decoder_embeddings = tf.keras.layers.Embedding(vocab_size,embed_size,
                                                  mask_zero=True)(decoder_inputs)
    decoder_embeddings = tf.keras.layers.Dropout(0.1)(decoder_embeddings)
    
    encoder = tf.keras.layers.Bidirectional(
        tf.keras.layers.LSTM(256,return_state=True,return_sequences=True,dropout=0.1)
    )
    encoder_outputs,forward_h,forward_c,backward_h,backward_c = encoder(encoder_embeddings)
    state_h = tf.keras.layers.Concatenate()([forward_h,backward_h])
    state_c = tf.keras.layers.Concatenate()([forward_c,backward_c])
    encoder_states = [state_h,state_c]
    
    decoder = tf.keras.layers.LSTM(512,return_state=True,return_sequences=True,dropout=0.1)
    decoder_outputs ,_,_ =decoder(decoder_embeddings,initial_state=encoder_states)
    
    attention_layer = tf.keras.layers.Attention()
    context_vector = attention_layer([decoder_outputs,encoder_outputs])
    decoder_combined = tf.keras.layers.Concatenate()([decoder_outputs,context_vector])
    
    dense_layer = tf.keras.layers.Dense(256,activation="relu")
    dense_output = dense_layer(decoder_combined)
    dense_output = tf.keras.layers.Dropout(0.1)(dense_output)
    
    output_layer = tf.keras.layers.Dense(vocab_size,activation="softmax")
    Y_proba = output_layer(dense_output)
    model = tf.keras.Model(inputs=[encoder_inputs, decoder_inputs], outputs=[Y_proba])
    return model

In [22]:
model = create_model()

initial_learning_rate = 0.001
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=10000,
    decay_rate=0.9,
    staircase=True
)
model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
    metrics=["accuracy"]
)

print(model.summary())


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 50)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 50, 128)   │    512,000 │ input_layer[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, 50)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 50, 128)   │          0 │ embedding[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal           │ (None, 50)        │          0 │ input_layer[0][0] │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, 50, 128)   │    512,000 │ input_layer_1[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional       │ [(None, 50, 512), │    788,480 │ dropout[0][0],    │
│ (Bidirectional)     │ (None, 256),      │            │ not_equal[0][0]   │
│                     │ (None, 256),      │            │                   │
│                     │ (None, 256),      │            │                   │
│                     │ (None, 256)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 50, 128)   │          0 │ embedding_1[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 512)       │          0 │ bidirectional[0]… │
│ (Concatenate)       │                   │            │ bidirectional[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 512)       │          0 │ bidirectional[0]… │
│ (Concatenate)       │                   │            │ bidirectional[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ [(None, 50, 512), │  1,312,768 │ dropout_1[0][0],  │
│                     │ (None, 512),      │            │ concatenate[0][0… │
│                     │ (None, 512)]      │            │ concatenate_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal_1         │ (None, 50)        │          0 │ input_layer_1[0]… │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attention           │ (None, 50, 512)   │          0 │ lstm_1[0][0],     │
│ (Attention)         │                   │            │ bidirectional[0]… │
│                     │                   │            │ not_equal_1[0][0… │
│                     │                   │            │ not_equal[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_2       │ (None, 50, 1024)  │          0 │ lstm_1[0][0],     │
│ (Concatenate)       │                   │            │ attention[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 50, 256)   │    262,400 │ concatenate_2[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 50, 256)   │          0 │ dense[0][0]     

 Total params: 4,415,648 (16.84 MB)

 Trainable params: 4,415,648 (16.84 MB)

 Non-trainable params: 0 (0.00 B)

None


In [24]:
history = model.fit((X_train_enc,X_train_dec),Y_train,
                   epochs = 10,
                   batch_size =16,
                   validation_data=((X_valid_enc,X_valid_dec),Y_valid))

Epoch 1/10


InvalidArgumentError: Graph execution error:

Detected at node functional_1/attention_1/sub defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start

  File "/usr/local/lib/python3.11/dist-packages/tornado/platform/asyncio.py", line 211, in start

  File "/usr/lib/python3.11/asyncio/base_events.py", line 608, in run_forever

  File "/usr/lib/python3.11/asyncio/base_events.py", line 1936, in _run_once

  File "/usr/lib/python3.11/asyncio/events.py", line 84, in _run

  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelbase.py", line 510, in dispatch_queue

  File "/usr/local/lib/python3.11/dist-